# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filename = 'city_weather.csv'
city_weather_file = os.path.join(os.getcwd(), f'output_data\\{filename}')
weather_df = pd.read_csv(city_weather_file)
weather_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Cloud,Country,Date Time
0,0,Avarua,-21.2078,-159.7750,75.25,53,3.44,75,CK,1634928516
1,1,Vitim,59.4511,112.5578,34.50,84,8.79,75,RU,1634928518
2,2,Calvinia,-31.4707,19.7760,50.07,76,15.88,4,ZA,1634928518
3,3,Dingle,10.9995,122.6711,75.40,93,5.82,100,PH,1634928519
4,4,Castro,-24.7911,-50.0119,73.27,44,2.82,2,BR,1634928519
...,...,...,...,...,...,...,...,...,...,...
570,570,Jiuquan,39.7432,98.5174,42.21,36,4.07,4,CN,1634928797
571,571,Grindavik,63.8424,-22.4338,45.07,93,25.32,90,IS,1634928798
572,572,Amahai,-3.3333,128.9167,77.52,85,3.13,98,ID,1634928798
573,573,Beidao,34.5686,105.8933,50.07,82,4.50,100,CN,1634928621


In [12]:
#drop null data
weather_df = weather_df.dropna()
weather_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Cloud,Country,Date Time
0,0,Avarua,-21.2078,-159.7750,75.25,53,3.44,75,CK,1634928516
1,1,Vitim,59.4511,112.5578,34.50,84,8.79,75,RU,1634928518
2,2,Calvinia,-31.4707,19.7760,50.07,76,15.88,4,ZA,1634928518
3,3,Dingle,10.9995,122.6711,75.40,93,5.82,100,PH,1634928519
4,4,Castro,-24.7911,-50.0119,73.27,44,2.82,2,BR,1634928519
...,...,...,...,...,...,...,...,...,...,...
570,570,Jiuquan,39.7432,98.5174,42.21,36,4.07,4,CN,1634928797
571,571,Grindavik,63.8424,-22.4338,45.07,93,25.32,90,IS,1634928798
572,572,Amahai,-3.3333,128.9167,77.52,85,3.13,98,ID,1634928798
573,573,Beidao,34.5686,105.8933,50.07,82,4.50,100,CN,1634928621


In [4]:
highest_humidity = weather_df['Humidity'].max().astype(float)
highest_humidity

100.0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lon' into  locations 
locations = weather_df[["Lat", "Lon"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
humidity = weather_df["Humidity"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=highest_humidity, 
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.
* Drop any rows will null values.

In [7]:
hotel_df = weather_df.loc[(weather_df['Temp'] <=80) & (weather_df['Temp'] >=70) & (weather_df['Wind Speed'] <=10) & (weather_df['Cloud'] == 0)]
hotel_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Cloud,Country,Date Time
65,65,Salalah,17.0151,54.0924,78.89,69,2.30,0,OM,1634928554
72,72,Trelew,-43.2490,-65.3051,70.57,11,9.95,0,AR,1634928557
100,100,Vao,-22.6667,167.4833,72.43,80,1.43,0,NC,1634928573
243,243,Bānsi,27.1833,82.9333,70.66,68,2.59,0,IN,1634928637
388,388,Torres,-29.3353,-49.7269,70.36,65,5.55,0,BR,1634928711
516,516,Awjilah,29.1081,21.2869,75.13,33,8.68,0,LY,1634928765


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotels = []
ht_lats = []
ht_lngs = []
for index, row in hotel_df.iterrows():
    g_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={row['Lat']}%2C{row['Lon']}&radius=5000&type=hotel&key={g_key}"
    try:
        response = requests.get(g_url, timeout=3)
        response.raise_for_status()
        if response.status_code == 200:
            response_json = response.json()
            hotels.append(response_json['results'][1]['name'])
            ht_lats.append(response_json['results'][1]['geometry']['location']['lat'])
            ht_lngs.append(response_json['results'][1]['geometry']['location']['lng'])
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
        logs.append(errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
        logs.append(errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
        logs.append(errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
        logs.append(err)
hotel_df["Hotel Name"] = hotels
hotel_df["Hotel Lat"] = ht_lats
hotel_df["Hotel Lng"] = ht_lngs
hotel_df

C:\Users\trang\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\trang\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\trang\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind Speed,Cloud,Country,Date Time,Hotel Name,Hotel Lat,Hotel Lng
65,65,Salalah,17.0151,54.0924,78.89,69,2.30,0,OM,1634928554,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel",17.019113,54.061124
72,72,Trelew,-43.2490,-65.3051,70.57,11,9.95,0,AR,1634928557,Hotel Touring Club,-43.250991,-65.306325
100,100,Vao,-22.6667,167.4833,72.43,80,1.43,0,NC,1634928573,Hotel Kou-Bugny,-22.659695,167.443914
243,243,Bānsi,27.1833,82.9333,70.66,68,2.59,0,IN,1634928637,Tripmate,27.171851,82.928322
388,388,Torres,-29.3353,-49.7269,70.36,65,5.55,0,BR,1634928711,Farol Hotel,-29.340857,-49.726705
516,516,Awjilah,29.1081,21.2869,75.13,33,8.68,0,LY,1634928765,مدرسة عبدالله بن أبي السرح أوجلة,29.108894,21.283804


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel</dd>\n<dt>City</dt><dd>Salalah</dd>\n<dt>Country</dt><dd>OM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Touring Club</dd>\n<dt>City</dt><dd>Trelew</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Kou-Bugny</dd>\n<dt>City</dt><dd>Vao</dd>\n<dt>Country</dt><dd>NC</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Tripmate</dd>\n<dt>City</dt><dd>Bānsi</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Farol Hotel</dd>\n<dt>City</dt><dd>Torres</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>مدرسة عبدالله بن أبي السرح أوجلة</dd>\n<dt>City</dt><dd>Awjilah</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n']

In [10]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='red',
    stroke_color='red', scale=6,
    info_box_content=hotel_info
)
# Add marker layer ontop of heat map

fig = gmaps.figure()
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))